Parameters: 
- width, height of NN
- image resolution. 
- grayscale. 
- drop out. (may increase interpretability of mid layers?)
- L1 vs L2 loss.

In [ ]:
import os
from functools import lru_cache
import matplotlib.pylab as plt
import numpy as np
from scipy.ndimage import zoom
from IPython import display
from imageio import imread

In [ ]:
import tensorflow as tf 
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras import Model

print("TF version:", tf.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

In [ ]:
try:
    from google.colab import drive
except ModuleNotFoundError:
    ROOT = 'images'
else:
    drive.mount('/content/drive')
    ROOT = '/content/drive/My Drive/Colab Notebooks/images'

In [ ]:
class MyModel(Model):
  def __init__(self, width, depth, n_channels = 3):
    super(MyModel, self).__init__()
    self.n_channels = n_channels
    self.myLayers = []
    for i in range(depth):
      layer = Dense(
        width, activation='relu', name = f'relu_layer_{i}', 
        kernel_initializer=tf.initializers.RandomNormal(
          stddev = (width** -.5), 
        ), 
        bias_initializer  =tf.initializers.RandomNormal(stddev=0.01),
      )
      self.myLayers.append(layer)
    self.last = Dense(n_channels, activation='sigmoid', name = f'sigmoid_layer')

  def call(self, x):
    for layer in self.myLayers:
      x = layer(x)
    return self.last(x)


In [ ]:
@lru_cache()
def getRaster(width, height):
  buffer = np.zeros((width*height, 2))
  x_lin_space = np.linspace(-1, 1, width)
  y_lin_space = np.linspace(-1, 1, height)
  for x in range(width):
    for y in range(height):
      buffer[x * height + y, :] = (x_lin_space[x], y_lin_space[y])
  return buffer

In [ ]:
def view(model, width, height, ax = plt, fig = None, view_h = 5):
    output = model.predict(getRaster(width, height))
    ax.imshow(
        np.reshape(output, (width, height, model.n_channels)), 
        vmin=0, vmax=1, 
    )
    ax.axis('off')
    (fig or plt.gcf()).set_size_inches(view_h / height * width, view_h)

In [ ]:
def viewInitField():
    model = MyModel(4, 4, 3)
    model.build((None, 2))
    model.summary()
    view(model)
# viewInitField()

In [ ]:
os.chdir(ROOT)
img_names = os.listdir()

In [ ]:
def loadData(img_name, resolution = 50, to_gray = False):
  img = imread(img_name) / 255
  width, height = img.shape[:2]
  try:
    if img.shape[2] == 4:
      img = img[:, :, :3]
    elif img.shape[2] == 1:
      raise IndexError
    else:
      assert img.shape[2] == 3
  except IndexError:
    t = np.zeros((width, height, 3))
    t[:, :, 0] = img
    t[:, :, 1] = img
    t[:, :, 2] = img
    img = t
  zoom_k = resolution / (width * height) ** .5
  tt = zoom(img[:, :, 0], zoom_k, order=1)
  width, height = tt.shape
  t = np.zeros((width, height, 3))
  t[:, :, 0] = tt
  t[:, :, 1] = zoom(img[:, :, 1], zoom_k, order=1)
  t[:, :, 2] = zoom(img[:, :, 2], zoom_k, order=1)
  img = t
  if to_gray:
    img = np.mean(img, axis=2)
    n_channels = 1
  else:
    n_channels = 3
  x = getRaster(width, height)
  y = np.zeros((width * height, n_channels))
  for i in range(n_channels):
    y[:, i] = np.reshape(img[:, :, i], (width * height, ))
  return x, y, width, height, img

In [ ]:
previewIter = iter(img_names)

In [ ]:
# Run this cell multiple times to preview all data. 
try:
  name = next(previewIter)
except StopIteration:
  print("No more.")
else:
  x, y, w, h, img = loadData(name, 150)
  print(name)
  plt.imshow(img)

In [ ]:
# class MyCallback(tf.keras.callbacks.Callback):
#     def __init__(self, width, height):
#         super().__init__()
#         self.width = width
#         self.height = height
#     def on_epoch_begin(self, epoch, logs=None):
#         print(epoch)
#         sleep(1)
#         view(model, self.width, self.height, 5)
#         display.clear_output(wait=True)
#         display.display(plt.gcf())
# #         sleep(.01)

In [ ]:
def train(
    img_name, resolution = 150, nn_width = 64, nn_depth = 3, 
    is_gray = False, loss = tf.keras.losses.mean_squared_error, 
):
    x, y, w, h, _ = loadData(img_name, resolution, is_gray)
    fig, axes = plt.subplots(2, 2)
    ax = axes[0][0]
    model = MyModel(nn_width, nn_depth, 1 if is_gray else 3)
    model.compile(
        optimizer='adam',
        loss=loss,
    )
    for epoch in range(40):
        model.fit(
            x, y, 
            steps_per_epoch = 8, 
            epochs = 1, 
            verbose = 0, 
        )
        view(model, w, h, ax, fig, 5)
        display.clear_output(wait=True)
        display.display(plt.gcf())
    display.clear_output(wait=True)
    view(model, w, h, ax, fig, 5)
    print("ok")
train(img_names[2])


In [ ]:
view(model, w, h)

Tutorial

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5)


In [ ]:
model.evaluate(x_test,  y_test, verbose=2)